<a href="https://colab.research.google.com/github/githubuserpk/gcp/blob/master/genai/bigquery_authentication_method1_using_credential_object.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#install vertex ai
!pip install google-cloud-aiplatform



In [9]:
from google.colab import auth as google_auth

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

In [10]:
#install vertex ai
!pip install google-cloud-aiplatform

#install vertex ai
!pip install google-cloud-aiplatform

#REGION='us-central1'
PROJECT_ID='pkdeltaai-01'

In [11]:
import vertexai
vertexai.init(project="pkdeltaai-01", credentials=creds) #, location="us-central1")

from google.cloud import bigquery
import pandas as pd

#This function takes a sql query string and runs it on bigquery

def run_bq_query(sql):

    # Create BQ client
    bq_client = bigquery.Client(project = PROJECT_ID) # location=REGION)

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True,
                                         use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql,
                                    job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

In [12]:
# define list of programming language tags we want to query

#language_list = ["python", "html", "r", "css"]
language_list = ["python"]

so_df = pd.DataFrame()

for language in language_list:

    print(f"generating {language} dataframe")

    query = f"""
    SELECT
        q.title
    FROM
        `bigquery-public-data.stackoverflow.posts_questions` q
    JOIN
        `bigquery-public-data.stackoverflow.posts_answers` a
    ON
        q.accepted_answer_id = a.id
    WHERE
        q.accepted_answer_id IS NOT NULL AND
        REGEXP_CONTAINS(q.tags, "python") AND
        a.creation_date >= "2020-01-01"
    LIMIT
        500
    """


    language_df = run_bq_query(query)
    language_df["category"] = language
    so_df = pd.concat([so_df, language_df],
                      ignore_index = True)


generating python dataframe
Finished job_id: 7b3f606c-6baf-499b-82f5-1a499ef42aeb


In [14]:
so_df

,title,category
0,Calling flask app from another flask app locally,python
1,Save a variable from a function to reuse when ...,python
2,Removing an element of a list,python
3,How to transfer items from QListWidget based i...,python
4,TC - how to assert on <img src=>,python
...,...,...
495,"Why is the ""else"" line giving an invalid synta...",python
496,Determine if each item in Series is in another...,python
497,Merging two or more dictionaries when they hav...,python
498,Extract Sentences from review column and addin...,python
